<a href="https://colab.research.google.com/github/Shahriarin2garden/fake-news/blob/main/FakeNewsDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

train=pd.read_csv("/content/Fake_train (1).csv")
train2=pd.read_csv("/content/Fake_dev.csv")

In [ ]:
!pip install langdetect
!pip install langid


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 17.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=d02708b2442594ec8e192c081462ebe200ef2acc2b53a4161f1ee9ab9eea6650
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 25.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941171 sha256=b43552cd82fdf8d5f44d72ae2e8317e11689f9fff15618e7ecc3adb489955707
  Stored in directory: /root/.cache/pip/wheels/32/6a/b6/b7eb43a6ad55b139c15c5daa29f3707659cfa6944d3c696f5b
Successfully built langid


In [ ]:
# from langdetect import detect, DetectorFactory
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
# Set seed for consistent results
# DetectorFactory.seed = 0

# Load dataset (replace `train` with your actual DataFrame)
data = train

# Handle missing or null values in the text column
data['text'] = data['text'].fillna('')

# Function to safely detect language
def safe_detect(text):
    try:
        # Skip detection for empty strings or strings with no alphanumeric characters
        if not text.strip() or not any(char.isalnum() for char in text):
            return "unknown"
        return detect(text)
    except Exception:
        return "error"  # Return "error" for texts that cause issues

# Apply language detection
data['language'] = data['text'].apply(safe_detect)

# Find unique languages detected
unique_languages = data['language'].unique()

# Output the unique languages
print("Unique languages detected:", unique_languages)


Unique languages detected: ['error' 'unknown']


In [ ]:
!pip install transformers torch scikit-learn pandas

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch


In [ ]:
# Check for empty texts
print(f"Before removing empty texts: {len(data)} rows")

# Filter out rows where the 'Text' column is empty or contains only whitespace
data = data[data['text'].str.strip().astype(bool)]
train2 = train2[train2['text'].str.strip().astype(bool)]

# Check the number of rows after removal
print(f"After removing empty texts: {len(data)} rows")


Before removing empty texts: 3257 rows
After removing empty texts: 3257 rows


In [ ]:
import re

# Function to remove emojis
def remove_emojis(text):
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U00002500-\U00002BEF"  # chinese char
        "\U00002702-\U000027B0"
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )





    -
    return emoji_pattern.sub(r"", text)


In [ ]:
# @title Default title text
def preprocess_text(text):
    # Remove emojis
    text = remove_emojis(text)
    # Lowercase the text
    text = text.lower()
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)
    # Remove special characters and punctuation
    text = re.sub(r"[^\w\s]", "", text)
    # Remove extra whitespace
    text = re.sub(r"\s+", " ", text).strip()
    return text


In [ ]:
# Apply preprocessing
data['text'] = data['text'].apply(preprocess_text)
train2['text'] = train2['text'].apply(preprocess_text)



In [ ]:
data['label'] = data['label'].map({'Fake': 0, 'original': 1})
# Encode labels
train2['label'] = train2['label'].map({'Fake': 0, 'original': 1})  # Encode labels

In [ ]:
data['label'].value_counts()

,count
label,
1,1658
0,1599


In [ ]:
from sklearn.utils import resample
import pandas as pd

# Assuming 'data' is your DataFrame and 'label' is the column with class labels.

# Separate classes
class_1 = data[data['label'] == 1]  # Majority class
class_0 = data[data['label'] == 0]  # Minority class

# Upsample minority class (class_0) to match class_1
class_0_upsampled = resample(class_0,
                             replace=True,  # Allow duplicates
                             n_samples=len(class_1),  # Match the majority class size
                             random_state=42)

# Combine both classes
balanced_data = pd.concat([class_1, class_0_upsampled])

# Shuffle the dataset
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Print the label counts to confirm balance
print(balanced_data['label'].value_counts())

# Optional: Save the balanced dataset to a new file
# balanced_data.to_csv('balanced_data.csv', index=False)


label
0    1658
1    1658
Name: count, dtype: int64


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

# Tokenize data
def tokenize_data(texts, labels=None):
    tokenized = tokenizer(list(texts), padding=True, truncation=True, max_length=512, return_tensors="pt")
    if labels is not None:
        tokenized['labels'] = torch.tensor(labels)
    return tokenized

In [ ]:
X_train=data['text']
X_test=train2['text']
y_train=data['label']
y_test=train2['label']

In [ ]:
train_encodings = tokenize_data(X_train, y_train)
test_encodings = tokenize_data(X_test, y_test)

In [ ]:
class FakeNewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        return {key: tensor[idx] for key, tensor in self.encodings.items()}

train_dataset = FakeNewsDataset(train_encodings)
test_dataset = FakeNewsDataset(test_encodings)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=2).to(device)


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Define model and tokenizer names
    models = {
        "IndicBERT": ("ai4bharat/indic-bert", "ai4bharat/indic-bert"),
        "mBERT": ("bert-base-multilingual-cased", "bert-base-multilingual-cased"),
        "XLM-RoBERTa": ("xlm-roberta-base", "xlm-roberta-base")  # Include the original model for comparison
    }

    # Train and evaluate each model
    results = {}
    for model_name, (model_path, tokenizer_path) in models.items():
        print(f"Training and evaluating {model_name}...")
        report = train_and_evaluate_model(model_path, tokenizer_path, train_dataset, test_dataset)
        results[model_name] = report

    # Print the results
    for model_name, report in results.items():
        print(f"--- {model_name} ---")
        print(report)  # This will print the classification report for each model


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',                 # Directory to save model checkpoints
    evaluation_strategy="epoch",           # Evaluate after every epoch
    save_strategy="epoch",                 # Save checkpoints after every epoch
    logging_dir='./logs',                  # Directory for logging
    learning_rate=5e-4,                    # Learning rate
    per_device_train_batch_size=8,         # Training batch size
    per_device_eval_batch_size=8,          # Evaluation batch size
    num_train_epochs=3,                    # Number of epochs
    weight_decay=0.01,                     # Weight decay
    load_best_model_at_end=True,           # Load the best model at the end
    logging_steps=10,                      # Log every 10 steps
    save_total_limit=1,                    # Keep only the best model checkpoint
    metric_for_best_model="eval_loss",     # Use evaluation loss to determine the best model
    greater_is_better=False                # Lower eval loss is better
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,0.691400,0.694274
2,0.740100,0.698050


Epoch,Training Loss,Validation Loss
1,0.691400,0.694274
2,0.740100,0.698050
3,0.685600,0.693656


TrainOutput(global_step=1224, training_loss=0.7110574876560884, metrics={'train_runtime': 1281.769, 'train_samples_per_second': 7.623, 'train_steps_per_second': 0.955, 'total_flos': 2570858121922560.0, 'train_loss': 0.7110574876560884, 'epoch': 3.0})

In [ ]:
predictions = trainer.predict(test_dataset)
y_pred = torch.argmax(torch.tensor(predictions.predictions), axis=1).numpy()
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       406
           1       0.50      1.00      0.67       409

    accuracy                           0.50       815
   macro avg       0.25      0.50      0.33       815
weighted avg       0.25      0.50      0.34       815



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.cuda.amp import autocast

# Define the paths
model_checkpoint = "/content/results/checkpoint-1224"
base_model = "xlm-roberta-base"  # Replace with the base model you used during training

# Load the tokenizer from the base model
tokenizer = AutoTokenizer.from_pretrained(base_model)

# Load the fine-tuned model from the best checkpoint
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)

# Load the test data
test_data = pd.read_csv("/content/Fake_test_without_labels.csv")  # Replace with your test data file
texts = test_data["text"].fillna("").tolist()  # Handle empty or NaN text entries

# Tokenize the test data
test_encodings = tokenizer(
    texts,
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt"
)

# Move the model to the device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Make predictions
model.eval()  # Set the model to evaluation mode

# Use mixed precision to reduce memory usage if on GPU
with torch.no_grad():
    predicted_labels = []
    for i in range(0, len(texts), 16):  # Change 16 to your batch size
        batch_texts = texts[i:i+16]
        batch_encodings = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )
        inputs = {key: val.to(device) for key, val in batch_encodings.items()}

        with autocast():  # Use mixed precision if on GPU
            outputs = model(**inputs)
            logits = outputs.logits
            batch_predictions = torch.argmax(logits, dim=1).cpu().numpy()  # Get predicted labels
            predicted_labels.extend(batch_predictions)

        # Empty cache after each batch if needed
        torch.cuda.empty_cache()

# Map predicted labels to "Fake" or "Original"
label_mapping = {0: "Fake", 1: "Original"}
test_data["label"] = [label_mapping[label] for label in predicted_labels]

# Save the results
test_data.to_csv("VEL_Tamil_task1.csv", index=False)

# Preview the results
print(test_data.head())


<ipython-input-41-9864ac077965>:50: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Use mixed precision if on GPU


        Id                                               text     label
0  Fake_01  5000 ഉള്ള പോൾ  ലോഗ്‌ഡ്‌വൻ ഇപ്പോള് 250000 എന്താ...  Original
1  Fake_02  ഓഷോ രജനീഷ്  പറഞ്ഞപോലെ  എനിക്കപ്പോൾ തോന്നിയത് അ...  Original
2  Fake_03  ചേട്ടാ  വാർത്ത  വയ്ക്കുന്നത്  കേരളത്തിലാണ്  സം...  Original
3  Fake_04             Shame for entire Woman&#39;s of Kerala  Original
4  Fake_05  135 code janaghal andhu wide business cheythal...  Original
